In [1]:
import openai
import os
import json
import time
import numpy as np

In [2]:
openai.api_key = ""

In [3]:
class PromptGenerator:
    def __init__(self,):
        self.summary_query = ['Summarize this paragraph into a sentence: ',
            'Brifely rephrase this paragraph: ',
            'Express this paragraph in a sentence: ',
            'Make a conclusion for this paragraph in a sentence: ',
            "Summarize the passage in a concise manner: ",
            "Condense the text into a brief summary: ",
            "Present a brief synopsis of the text: ",
            "Sum up the passage succinctly: ",]
        self.polish_query = ['Polish this text: ',
            'Rewrite this abstract without changing the semantics: ',
            'Please rephrase this text: ',
            'Express this text differently: ',
            'Refine this paragraph:',
            'Make this paragraph more polished:',
            'Spruce up this paragraph:',
            'Please revise this text:']
    def prompt_gen(self,text):
        return 'Generate an abstract for the title: ' + text
    def prompt_sum(self,text,i):
        return self.summary_query[i] + text
    def prompt_re(self,text):
        return self.polish_query[2] + text

In [4]:
def InsistAsk(text):
    while True:
        try:
            response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
                {"role": "user", "content": text},
            ]
        )
            text = response['choices'][0]['message']['content']
            text = text.strip()
            return text
        except:
            print('sleep')
            time.sleep(10)
    

In [5]:
Prompter = PromptGenerator()

In [6]:
root_path = '../ExampleData/'
dataset = 'PubMed'
mix_ratio = 0.3
DataNum = 500

path = os.path.join(root_path,dataset)
with open(os.path.join(path,'raw.txt')) as f:
    lines = f.readlines()

In [ ]:
for i in range(DataNum):
    raw_data = json.loads(lines[i].strip())
    response = {'title':' '.join(raw_data['title']).lower(),'abstract':' '.join(raw_data['abs']).lower()}

    response['gen_abstract'] = InsistAsk(Prompter.prompt_gen(response['title'])).lower()
    time.sleep(3)
    
    response['pol_abstract'] = InsistAsk(Prompter.prompt_re(response['abstract'])).lower()
    time.sleep(3)
    
    sentences = response['abstract'].split('.')
    sen_num = len(sentences)-1
    max_selection_num = int(mix_ratio*sen_num) + 1
    if max_selection_num > 1:
        human_sentence_num = np.random.randint(1,max_selection_num)
    else:
        human_sentence_num = 0
    seleteced_sids = np.random.permutation(sen_num)[:human_sentence_num]
    seleteced_sids = seleteced_sids[seleteced_sids.argsort()].tolist() + [sen_num]
    pointer = 0
    mix_sentence = ''
    for right in seleteced_sids:
        text_to_polish = sentences[pointer:right]
        text_to_retain = sentences[right]
        text_to_polish = '.'.join(text_to_polish)
        pointer = right + 1
        if not text_to_polish == '':
            text_to_polish = InsistAsk(Prompter.prompt_re(text_to_polish)).lower()
            time.sleep(5)
        mix_sentence += text_to_polish + ' ' + text_to_retain + ' '
    
    mix_sentence = mix_sentence.strip()
    
    response['mix_abstract'] = mix_sentence.lower()
    
    with open(os.path.join(path,'exp-data.txt'),'a') as f:
        f.write(json.dumps(response)+'\n')
        
    print(i)

sleep
0
sleep
sleep
sleep
sleep
sleep
